In [1]:
import pandas as pd
from gtfparse import read_gtf
from sklearn.cluster import DBSCAN
from collections import defaultdict
import sys
import scanpy as sc
sys.path.append('/data_nfs/og86asub/netmap/netmap-evaluation/')

from netmap.src.masking.external import get_genome_annotation_from_gtf, preprocess_bed_file, get_regulators

In [2]:
# 1. Load GTF file (replace path with your downloaded file)
gtf_path = "/data_nfs/datasets/GRCh38/Homo_sapiens.GRCh38.109.gtf"

window = 10000
bed_file = '/data_nfs/og86asub/netmap/netmap-evaluation/data/annotation/remap2022_crm_macs2_hg38_v1_0.bed'


gtf_df = read_gtf(gtf_path)

# Filter gene locations from gtf annotation file.
genes = get_genome_annotation_from_gtf(gtf_df)
tf_to_gene = preprocess_bed_file(bed_file, genes)



INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'ccds_id', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version', 'transcript_support_level']


In [3]:
tf_to_gene_df = get_regulators(tf_to_gene, genes, window=window)


In [4]:
grn_adata = sc.read_h5ad('/data_nfs/og86asub/netmap/netmap-evaluation/results_old/netmap/config_1/config_easy/net_53_1113_net_70_1328_net_84_59925/grn_lrp.h5ad')

In [5]:
grn_adata.var = grn_adata.var.merge(tf_to_gene_df.loc[:, ['edge', 'regulator']], left_on='index', right_on='edge', how='left')
grn_adata.var.regulator = grn_adata.var.regulator.apply(lambda x: False if pd.isna(x) else True)

In [8]:
grn_adata.var

,source,target,edge,regulator
0,ABCB11,ABCB11,NaN,False
1,ABCD3,ABCB11,NaN,False
2,ADAMTS4,ABCB11,NaN,False
3,ADH1B,ABCB11,NaN,False
4,ADH1C,ABCB11,NaN,False
...,...,...,...,...
42844,ZFAT,ZXDC,NaN,False
42845,ZFHX4,ZXDC,NaN,False
42846,ZNF202,ZXDC,NaN,False
42847,ZNF461,ZXDC,NaN,False
